# Segmentation EDA

## 데이터 다운로드 후 압축풀기

In [0]:
# 데이터를 다운로드 합니다. 
!wget https://pai-datasets.s3.ap-northeast-2.amazonaws.com/vision_multi_campus/images_labels.zip

# 압축을 풉니다.
import zipfile
zip_obj = zipfile.ZipFile('./images_labels.zip')
zip_obj.extractall('./')
zip_obj.close()

### 데이터 구성

````
images_labels/images/
                |- *.jpg
                |- *.jpg
             /labels/car
                    |- hyundai.json
````


In [0]:
import os
import cv2
import json
import glob
import pprint
import numpy as np 
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils import to_categorical

## Images / Labels 불러오기 

데이터가 들어있는 images_labels폴더에서 모든 이미지 경로들을 불러옵니다

In [0]:
# 모든 이미지 경로를 불러 옵니다. 
image_dir = './images_labels/images'
img_paths = glob.glob('{}/GH*.jpg'.format(image_dir))
print(len(img_paths))

# 샘플 이미지를 불러옵니다. 
img = Image.open(img_paths[0])
plt.imshow(img)

# Label 을 불러 옵니다
car_label = './images_labels/labels/car/hyundai.json'
f = open(car_label, 'r')
car_jsons = json.load(f)

In [0]:
def segmentation_info(jsons_info, ind, show=True):
    """
    Description:
        해당 함수는 index 을 입력받아 image_name 과 image 번호와 
        그리고 annotation 정보를 반환합니다. 
        index 번호는 jsons_info['images']에 들어있는 객체의 index 번호를 말합니다. 

    Args:
        jsons_info: dict, json 파일에서 읽어온 정보 
        ind: int, index 정보 
    Return:
        image_name: str, 해당 index 에 해당하는 이미지 이름
        image_id: int, 해당 index 에 해당하는 이미지 번호 
        ret_annos: ndarray, 해당 index에 해당하는 annotation(polyline) 정보 
    """
    # Search Image id / name 
    image_name = jsons_info['images'][ind]['file_name']
    image_id = jsons_info['images'][ind]['id']

    # Load Segmentation Annotation
    ret_names = [] 
    ret_ids = []
    ret_annos = []
    for anno in jsons_info['annotations']:
        if anno['image_id'] == image_id:
            seg_anno = anno['segmentation']

            sample_anno = np.reshape(seg_anno, [1, -1, 2])
            sample_anno = sample_anno.astype(np.int64)
            
            # Showing Image 
            if show:
                sample_img = np.array(Image.open(os.path.join(image_dir, image_name)))
                cv2.polylines(sample_img, [sample_anno], True, (255,0,0))
                print(image_name, image_id)
                plt.figure(figsize=(30,10))
                plt.imshow(sample_img)
                plt.show()
                
            ret_annos.append(sample_anno)
            
    return image_name, image_id, ret_annos

segmentation_info(car_jsons, 150)

## EDA

In [0]:
import os
import cv2
import json
import glob
import pprint
import numpy as np 
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils import to_categorical

In [0]:
# 총 이미지 갯수 
n_images = len(car_jsons['images']) 
print('사용할 이미지 갯수 : {}'.format(n_images))

# 모든 이미지의 annotation 정보를 저장합니다. 
global_seg_annos = [] # [[seg_anno], [seg_anno1], [seg_anno2] ...]
global_loc_annos = [] # [[loc_anno], [loc_anno1], [seg_anno2] ...]
for ind in tqdm(range(n_images)[:]):
    
    # Extract Segmentation Annotation Info 
    img_name, img_id, annos = segmentation_info(car_jsons, ind, False)

    # Mask 갯수가 0이 아니면 Mask, Label을 만들고 저장한다. 
    if len(annos) !=0 :
        for anno in annos:
            anno = np.reshape(anno, [-1, 2])

            # 위 segmentation annotation 정보를 이용해 x1y1, x2y2 좌표를 찾아냅니다. 
            x1y1 = # fix me!
            x2y2 = # fix me!
            detect_anno = # fix me!
 
            global_loc_annos.append(detect_anno)
        global_seg_annos.extend(annos)

# 총 객체 갯수 
print('\n 모든 이미지 안에 들어 있는 객체 갯수 : {}'.format(len(global_loc_annos)))


### hs | ws Histogram

In [0]:
global_loc_annos = np.asarray(global_loc_annos)

# Image 내 모든 객체의 ws, hs
ws = # fix me!
hs = # fix me!

plt.hist(ws, alpha=0.5, label='ws')
plt.hist(hs, alpha=0.5, label='hs')
plt.legend()

### Ratios 

In [0]:
# visualization hs / ws ratios
ratios = hs/ws
plt.hist(hs/ws)

### Image Std mean 

In [0]:
"""
이미지를 한장씩 불러와 평균, 분산을 계산합니다. 
R,G,B channel 따로 따로 분산과 평균을 계산합니다.
"""

# 모든 이미지의 각 channel 평균, std 구하기
img_means = []
img_stds = []
for path in tqdm(img_paths):
    img = Image.open(path)
    # fix me! 이미지의 평균을 구해 img_means에 넣어주세요!
    # fix me! 이미지의 std를 구해 img_stds에 넣어주세요!

### Object Std mean

In [0]:
def generation_mask(image_size, polys, color):
    mask = np.zeros_like(image)
    for poly in polys:
        mask = cv2.fillPoly(mask, [poly], color)
    return mask

In [0]:
n_images = len(car_jsons['images']) 

rgb_means =[] 
rgb_stds =[] 
pixels = []

for ind in tqdm(range(n_images)[:]):
    
    # Extract Segmentation Annotation Info 
    img_name, img_id , annos = segmentation_info(car_jsons, ind, False)

    # Mask 갯수가 0이 아니면 Mask, Label을 만들고 저장한다. 
    if len(annos) !=0 :
        # Image Load 
        img = np.array(Image.open(os.path.join(image_dir, img_name)))
        
        # Generate binart Mask 
        mask = generation_mask(img_size, annos, [255,0,0])
        mask =  # fix me!
        indices2d = #fix me!

        # 픽셀 수 
        n_pixels = # fix me!
        # fix me! 위에서 만든 n_pixels를 위에서 만든 pixels에 넣어주세요
        
        # color
        # fix me! object의 rgb mean을 구해주세요
        # fix me! object의 rgb std를 구해주세요
        # fix me! rgb mean을 위에서 만든 rgb means에 넣어주세요
        # fix me! rgb std를 위에서 만든 rgb_stds에 넣어주세요

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---